# Agent Sandbox

In [40]:
import os

import nest_asyncio
from langfuse.llama_index import LlamaIndexCallbackHandler
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.agent import ReActAgent
from llama_index.core.callbacks import CallbackManager
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from rich import print

nest_asyncio.apply()

In [2]:
langfuse_callback_handler = LlamaIndexCallbackHandler(
    public_key="pk-lf-363e8a50-c2ef-4622-b47b-0fd9db3b90eb",
    secret_key=os.environ["LANGFUSE_SECRET_KEY"],
    host="http://localhost:3000",
)
Settings.callback_manager = CallbackManager([langfuse_callback_handler])

In [8]:
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)


def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

In [ ]:
# MODEL = "llama3.2:1b"  #        90 TPS     errors out
# MODEL = "deepseek-r1:1.5b"  #   80 TPS
# MODEL = "gemma2:2b"  #          60 TPS
# MODEL = "llama3.2:3b"         # 50 TPS    works...okay
# MODEL = "mistral:7b"          # 30 TPS    struggling...
# MODEL = "deepseek-r1:7b"      # 27 TPS    doesn't call tools
# MODEL = "llama3.1:8b"         # 24 TPS    works...okay
MODEL = "gemma2:9b"  #          19 TPS    works well!
# MODEL = "phi4:14b"  #           11 TPS    works well!
# MODEL = "deepseek-r1:14b"     # 13 TPS
# MODEL = "llama3.3:70b"        # ?? TPS

EMBEDDER = "all-minilm"  # 384
# EMBEDDER = "nomic-embed-text" # 768
# EMBEDDER = "bge-m3" # 1024
# EMBEDDER = MODEL

llm = Ollama(model=MODEL, request_timeout=120.0)
# Settings.llm = llm
Settings.embed_model = OllamaEmbedding(model_name=EMBEDDER)

agent = ReActAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True,
)

# print(llm)
response = agent.chat(
    # "What is 20+(2*4)? Use a tool to calculate every step.",
    "What is 2087213784617864+(284757845*4812346789764)? Use a tool to calculate every step.",
)
print(response)

In [37]:
documents = SimpleDirectoryReader("../data/llama_index/").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query(
    "What was the total amount of the 2023 Canadian federal budget?"
)
print(response.response)

In [41]:
budget_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="canadian_budget_2023",
    description="A RAG engine with some basic facts about the 2023 Canadian federal budget.",
)
agent = ReActAgent.from_tools([multiply_tool, add_tool, budget_tool], verbose=True)

In [ ]:
response = agent.chat(
    "What is the total amount of the 2023 Canadian federal budget multiplied by 3? Go step by step, using a tool to do any math."
)

print(response)